# Extend container image to mount blob storage using Blobfuse

In this example we extend an Azure-curated container image to mount an blob storage container. We then deploy the container as part of a managed online deployment to run an inference server using a trained model stored in the blob container. 

## Prerequisites

* To use Azure Machine Learning, you must have an Azure subscription. If you don't have an Azure subscription, create a free account before you begin. Try the [free or paid version of Azure Machine Learning](https://azure.microsoft.com/free/).

* Install and configure the [Python SDK v2](sdk/setup.sh).

* You must have an Azure resource group, and you (or the service principal you use) must have Contributor access to it.

* You must have an Azure Machine Learning workspace. 

* You must have an Azure Secure Container registry. One is created automatically created for a workspace without one upon first usage, however in this example we explicitly reference the container registry by name, so you need it beforehand. You can create one through the Azure Portal.

## Initial set up

### Set parameters

In [1]:
subscription_id = '<YOUR_SUBSCRIPTION_ID>'
resource_group = '<YOUR_RESOURCE_GROUP>'
workspace = '<YOUR_WORKSPACE>'
container_registry_name = '<YOUR_CONTAINER_REGISTRY>'
storage_account_name = '<YOUR_STORAGE_ACCOUNT_NAME>'

base_path = 'resources/sample3/'

In [2]:
subscription_id = '6fe1c377-b645-4e8e-b588-52e57cc856b2'
resource_group = 'v-alwallace-test'
workspace = 'valwallace'
container_registry_name = 'valwallaceskr'
existing_storage_account_name = 'valwallacestorage'

### Get clients

In [3]:
import os
from azure.ml import MLClient
from azure.storage.blob import BlobClient, BlobSasPermissions,BlobServiceClient, ContainerSasPermissions, generate_container_sas
from azure.mgmt.storage.v2021_09_01 import StorageManagementClient
from azure.mgmt.storage.v2021_09_01.models import StorageAccountCreateParameters, Sku, FileShare, BlobContainer
from azure.mgmt.containerregistry.v2021_12_01_preview import ContainerRegistryManagementClient
from azure.mgmt.authorization.v2021_03_01_preview import AuthorizationManagementClient
from azure.ml.entities import ManagedOnlineDeployment, ManagedOnlineEndpoint
from azure.identity import DefaultAzureCredential,InteractiveBrowserCredential
from random import randint

ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)
storage_client = StorageManagementClient(DefaultAzureCredential(),subscription_id)
cr_client = ContainerRegistryManagementClient(DefaultAzureCredential(),subscription_id)
auth_client = AuthorizationManagementClient(DefaultAzureCredential(),subscription_id)

## Build a Dockerfile with Blobfuse
Blobfuse is an Azure-supported FUSE driver that enables compute instances to mount blob and file share resources.

### Prepare the Blobfuse configuration file fuse_connection.cfg

```cfg
accountName ACCOUNTNAME
accountKey SASTOKEN
containerName CONTAINERNAME
``` 

#### Build a Dockerfile with Blobfuse
We begin with the no

#### Complete Dockerfile example

```Dockerfile 
FROM mcr.microsoft.com/azureml/minimal-ubuntu18.04-py37-cpu-inference:latest

USER root 

RUN  apt-get update
RUN  apt-get install -y wget apt-utils 
RUN  wget https://packages.microsoft.com/config/ubuntu/18.04/packages-microsoft-prod.deb
RUN  dpkg -i packages-microsoft-prod.deb 
RUN  apt-get remove -y wget 
RUN  apt-get update 
RUN  apt-get install -y --no-install-recommends fuse blobfuse libcurl3-gnutls libgnutls30 
RUN  rm -rf /var/lib/apt/lists/*

ENV BFMOUNT=/mnt/blobfuse
ENV BFCONFMOUNT=/etc/blobfuse
RUN mkdir 
RUN mkdir ${BFMOUNT} && chmod 0755 ${BFMOUNT} 

RUN 

ENTRYPOINT blobfuse ${BFMOUNT} --use-https=true --config-file=/home/fuse_connection.cfg --tmp-path=/tmp/blobfuse/tmp -o allow_other
```

### Build and push the Dockerfile to ACR

In [ ]:
!az login 
!az acr login --name {container_registry_name}
!docker build --secret id=FUSECONF,src=fuse_connection.cfg -t  {container_registry_name}.azurecr.io/storage-client . 

### asdf

### Create an endpoint

In [13]:
import azure.ml
azure.ml.entities.Data()

Data({'type': 'uri_folder', 'is_anonymous': True, 'auto_increment_version': False, 'name': '3e7fad21-dcaf-4b31-b2ff-3ef9a4d3093d', 'description': None, 'tags': {}, 'properties': {}, 'id': None, 'base_path': './', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f0c7fee7a90>, 'version': '1', 'latest_version': None, 'path': None})

In [18]:
blob = ml_client.datastores.get_default()
azure.ml.entities.Dataset()

Dataset({'is_anonymous': True, 'auto_increment_version': False, 'name': '5ab3ebd2-bbcb-40c6-a895-a200756443ad', 'description': None, 'tags': {}, 'properties': {}, 'id': None, 'base_path': './', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f0c7f502990>, 'version': '1', 'latest_version': None, 'paths': [], 'local_path': None})

In [36]:
envs = list(ml_client.environments.list())

In [40]:
envs[0].

In [ ]:
from random import randint
# Required
deployment_name = 'docker-storage'
container_name = 'docker-storage'
# Optional
endpoint_name = f'docker-storage-{randint(1e3,1e7)}'

endpoint = ManagedOnlineEndpoint(name=endpoint_name)
ml_client.online_endpoints.begin_create_or_update(endpoint, local=True)

# Check endpoint status
print(f'Endpoint Status: {endpoint.provisioning_state}')